In [1]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 33.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [8]:
import stanza
stanza.download("hi")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: hi (Hindi) ...
INFO:stanza:File exists: /root/stanza_resources/hi/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [11]:
from tensorflow.keras.models import load_model
from objs import max_len, pred_map, basic_pos_to_vector, pos_to_basic_pos
import numpy as np
import string
import stanza
import spacy
import logging


def is_hindi(token):
    return all("\u0900" <= char <= "\u097F" for char in token)

def get_pos_tags(sentence):

    st_logger = logging.getLogger("stanza")
    st_logger.setLevel(logging.ERROR)

    nlp_en = spacy.load("en_core_web_sm")
    nlp_hi = stanza.Pipeline("hi", download_method=None)

    tokens = sentence.split()
    tokens = [token.strip(string.punctuation + chr(2404)) for token in tokens] # chr(2404) = Hindi fullstop (।)
    pos_sequence = []

    for token in tokens:
        if is_hindi(token):
            doc = nlp_hi(token)
            pos_sequence.append(pos_to_basic_pos.get(doc.sentences[0].words[0].upos, "other"))
        else:
            doc = nlp_en(token)
            pos_sequence.extend([pos_to_basic_pos.get(token.pos_, "other") for token in doc])

    del nlp_en, nlp_hi

    return pos_sequence

def predict(input):

    pos_sequence = get_pos_tags(input)

    vectorized_pos_seq = [basic_pos_to_vector[token] for token in pos_sequence]

    if len(vectorized_pos_seq) < max_len:
        padding = [[0] * 9] * (max_len - len(vectorized_pos_seq))
        vectorized_pos_seq += padding
    else:
        vectorized_pos_seq = vectorized_pos_seq[:max_len]

    inp = np.array([vectorized_pos_seq])

    model = load_model("dominant_language_model_2.keras")
    pred = list(model.predict(inp, verbose=0)[0])

    return pred_map[pred.index(max(pred))]


# predict("यह जगह बहुत essential है।")

In [12]:
predict("यह जगह बहुत essential है।")

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


'Hindi'